In [39]:
# --- 1. 라이브러리 설치 및 임포트 ---

import pandas as pd
import numpy as np
import os
from IPython.display import display
from sklearn.neighbors import BallTree

In [40]:
# --- 2. 경로 설정 및 데이터 로드 ---

# 청크 단위 처리를 위한 상수 정의
RADIUS_EARTH_KM = 6371.0
CHUNK_SIZE      = 100_000

# 학습 데이터 파일 경로
RAW_DIR         = '../../data/raw'
OUT_DIR         = '../../data/processed/transportation-features'
TRAIN_RAW       = os.path.join(RAW_DIR, 'train.csv')
TEST_RAW        = os.path.join(RAW_DIR, 'test.csv')
SUBWAY_RAW      = os.path.join(RAW_DIR, 'subway_feature.csv')
BUS_RAW         = os.path.join(RAW_DIR, 'bus_feature.csv')

# 출력 파일 경로 설정
TRAIN_OUT       = os.path.join(OUT_DIR, 'train_transportation_features.csv')
TEST_OUT        = os.path.join(OUT_DIR, 'test_transportation_features.csv')

os.makedirs(OUT_DIR, exist_ok=True)

In [41]:
# --- 3. 정류장·역 데이터 로드 + BallTree 생성 ---

print("데이터를 로드합니다...")

try:
    subway_df = pd.read_csv(SUBWAY_RAW, usecols=['위도','경도'], dtype='float32')
    bus_df    = pd.read_csv(BUS_RAW,    usecols=['Y좌표','X좌표'], dtype='float32')

    # 위도·경도 → 라디안
    subway_rad = np.deg2rad(subway_df[['위도','경도']].values)
    bus_rad    = np.deg2rad(bus_df[['Y좌표','X좌표']].values)

    subway_tree = BallTree(subway_rad, metric='haversine')
    bus_tree    = BallTree(bus_rad,    metric='haversine')

    print("모든 데이터 로드 완료.")
    
except FileNotFoundError as e:
    print(f"오류: 파일 로드에 실패했습니다. 경로를 확인해주세요. \n>> {e}")

데이터를 로드합니다...
모든 데이터 로드 완료.


In [42]:
# --- 4. 원하는 컬럼 순서 정의 ---

col_order = [
    '지하철최단거리',
    '반경_1km_지하철역_수',
    '반경_500m_지하철역_수',
    '반경_300m_지하철역_수',
    '버스최단거리',
    '반경_1km_버스정류장_수',
    '반경_500m_버스정류장_수',
    '반경_300m_버스정류장_수',
]

In [43]:
# --- 4. 파생 피처 계산 함수 ---

def compute_transport_features(df):
    coords_rad = np.deg2rad(df[['좌표Y','좌표X']].values)

    # 1) 최단거리
    dist_sub, _ = subway_tree.query(coords_rad, k=1)
    dist_bus,  _ = bus_tree.query(coords_rad,    k=1)

    out = pd.DataFrame({
        '지하철최단거리': dist_sub.flatten() * RADIUS_EARTH_KM,
        '버스최단거리':    dist_bus.flatten() * RADIUS_EARTH_KM,
    }, index=df.index)

    # 2) 반경 내 개수 (‘1km’, ‘500m’, ‘300m’)
    radii = [(1.0, '1km'), (0.5, '500m'), (0.3, '300m')]
    for km, label in radii:
        rad = km / RADIUS_EARTH_KM
        cnt_sub = subway_tree.query_radius(coords_rad, rad, count_only=True)
        cnt_bus = bus_tree.query_radius(coords_rad,    rad, count_only=True)
        out[f'반경_{label}_지하철역_수']    = cnt_sub
        out[f'반경_{label}_버스정류장_수'] = cnt_bus

    return out

In [44]:
# --- 5-1) Train 데이터: 청크 단위 처리 및 미리보기 ---

train_parts = []

for chunk in pd.read_csv(
        TRAIN_RAW,
        usecols=['아파트명','좌표X','좌표Y'],
        dtype={'아파트명':'string','좌표X':'float32','좌표Y':'float32'},
        chunksize=CHUNK_SIZE
    ):
    feats = compute_transport_features(chunk.rename(columns={'좌표X':'좌표X','좌표Y':'좌표Y'}))
    feats.insert(0, '아파트명', chunk['아파트명'].values)
    # 컬럼 순서 맞추기
    feats = feats[['아파트명'] + col_order]
    train_parts.append(feats)

train_features_df = pd.concat(train_parts, ignore_index=True)

display(train_features_df.head(10))

,아파트명,지하철최단거리,반경_1km_지하철역_수,반경_500m_지하철역_수,반경_300m_지하철역_수,버스최단거리,반경_1km_버스정류장_수,반경_500m_버스정류장_수,반경_300m_버스정류장_수
0,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
1,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
2,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
3,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
4,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
5,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
6,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
7,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
8,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
9,개포6차우성,1.129536,0,0,0,0.060787,58,13,7


In [45]:
# --- 5-2) Test 데이터: 전체 처리 및 미리보기 ---

test_df = pd.read_csv(
    TEST_RAW,
    usecols=['아파트명','좌표X','좌표Y'],
    dtype={'아파트명':'string','좌표X':'float32','좌표Y':'float32'}
)

test_features_df = compute_transport_features(test_df)
test_features_df.insert(0, '아파트명', test_df['아파트명'].values)
test_features_df = test_features_df[['아파트명'] + col_order]

display(test_features_df.head(10))

,아파트명,지하철최단거리,반경_1km_지하철역_수,반경_500m_지하철역_수,반경_300m_지하철역_수,버스최단거리,반경_1km_버스정류장_수,반경_500m_버스정류장_수,반경_300m_버스정류장_수
0,개포6차우성,1.129536,0,0,0,0.060787,58,13,7
1,개포더샵트리에,0.306197,5,1,0,0.125649,72,15,8
2,개포우성3차,0.413784,4,1,0,0.098428,77,19,9
3,개포우성3차,0.413784,4,1,0,0.098428,77,19,9
4,개포우성3차,0.413784,4,1,0,0.098428,77,19,9
5,개포주공5단지,0.222354,4,1,1,0.131533,59,22,11
6,개포주공6단지,0.250276,4,1,1,0.094651,62,20,11
7,개포주공6단지,0.250276,4,1,1,0.094651,62,20,11
8,개포주공6단지,0.250276,4,1,1,0.094651,62,20,11
9,래미안블레스티지,0.887415,1,0,0,0.049742,49,12,6


In [46]:
# --- 6. 결과 저장 ---

# 파생변수 데이터프레임이 정상적으로 생성되었는지 확인 후 저장
if train_features_df is not None:
    try:
        # 파생변수 DataFrame을 CSV로 저장
        train_features_df.to_csv(TRAIN_OUT, index=False)
        print(f"🎉 train.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '{TRAIN_OUT}'")

    except Exception as e:
        print(f"\n오류: 파일 저장 중 문제가 발생했습니다. \n>> {e}")
else:
    print("저장할 파생변수 데이터가 없습니다. (셀 1)을 먼저 실행해주세요.")
    

if test_features_df is not None:
    try:
        # 파생변수 DataFrame을 CSV로 저장
        test_features_df.to_csv(TEST_OUT, index=False)
        print(f"🎉 test.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '{TEST_OUT}'")

    except Exception as e:
        print(f"\n오류: 파일 저장 중 문제가 발생했습니다. \n>> {e}")
else:
    print("저장할 파생변수 데이터가 없습니다. (셀 1)을 먼저 실행해주세요.")

🎉 train.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '../../data/processed/transportation-features/train_transportation_features.csv'
🎉 test.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '../../data/processed/transportation-features/test_transportation_features.csv'
